# `System.Collections.Generic.Dictionary<TKey, TValue>` and Javascript’s `Map`

Without ado, here is the punchline:

>You can translate from a C♯ `Dictionary<TKey, TValue>` to a Javascript `Map` [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Map)] via the `Object.entries` method [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Object/entries)].

The statement above becomes important when you are faced with passing an instance of `Dictionary<TKey, TValue>` from an ASP.NET server to a JavaScript client. Let us start with a dictionary:

In [1]:
var data = new Dictionary<string, string>
{
    {"key-one", "hello,42.3,2023-10-08T04:54:48Z"},
    {"key-two", "greetings,77.07,2023-10-13T02:39:42Z"},
    {"key-three", "ohayo,1024,2023-06-26T23:45:01Z"},
};

The Microsoft defaults will serialize this `data` into the following:

In [2]:
using System.Text.Json;

JsonSerializer.Serialize(data, new JsonSerializerOptions { WriteIndented = true })

{
  "key-one": "hello,42.3,2023-10-08T04:54:48Z",
  "key-two": "greetings,77.07,2023-10-13T02:39:42Z",
  "key-three": "ohayo,1024,2023-06-26T23:45:01Z"
}

In the world of JavaScript, the JSON above parses into an object with weird property names:

In [3]:
#!javascript

const json = `
{
    "key-one": "hello,42.3,2023-10-08T04:54:48Z",
    "key-two": "greetings,77.07,2023-10-13T02:39:42Z",
    "key-three": "ohayo,1024,2023-06-26T23:45:01Z"
}`;

const o = JSON.parse(json);

console.log(o);

console.log(o['key-two'])

greetings,77.07,2023-10-13T02:39:42Z

We can see that properties are so weird they cannot be access like this: `o.key-two`.

## turning our JavaScript object into a JavaScript `Map`

We may tempted to try to survive in the wilds of JavaScript with our object above, `o`, in spite of its weird property names. But when we ask an innocent newbie question like, “Is there a Javascript equivalent of the .NET Dictionary?” We realize, yes, there is the `Map` [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Map)]. The next newbie question is, “Can we feed an object to instantiate a `Map`?”

In [4]:
#!javascript

const json = `
{
    "key-one": "hello,42.3,2023-10-08T04:54:48Z",
    "key-two": "greetings,77.07,2023-10-13T02:39:42Z",
    "key-three": "ohayo,1024,2023-06-26T23:45:01Z"
}`;

const o = JSON.parse(json);

const data = new Map(o);

Error: object is not iterable (cannot read property Symbol(Symbol.iterator))

It looks like the answer is _no_.

When we see [the documentation](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Map/Map) for the `Map` constructor it becomes clear that we can only feed a `Map` an _iterable_ [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Iteration_protocols)], like an array. A determined, self-confident newbie would not completely collapse out of egocentric exhaustion and would dare to ask, “Can we decompose an object into an array?”

According to a StackOverflow [answer](https://stackoverflow.com/a/55399661/22944), the answer is _yes_:

>Use `Object.entries()` to get both the key and values, then give that to `Map()` to allow you to select your values by key.

In [5]:
#!javascript

const json = `
{
    "key-one": "hello,42.3,2023-10-08T04:54:48Z",
    "key-two": "greetings,77.07,2023-10-13T02:39:42Z",
    "key-three": "ohayo,1024,2023-06-26T23:45:01Z"
}`;

const o = JSON.parse(json);

const a = Object.entries(o);

console.log('The array:', a);

const data = new Map(a);

data.forEach((value, key, map) =>
{
    console.log(`map has key \`${key}\` [${map.has(key)}] with value: \`${value}\` `);
});

console.log('The `Map.get()` syntax:', data.get('key-two'));

The array:

map has key `key-one` [true] with value: `hello,42.3,2023-10-08T04:54:48Z` 

map has key `key-two` [true] with value: `greetings,77.07,2023-10-13T02:39:42Z` 

map has key `key-three` [true] with value: `ohayo,1024,2023-06-26T23:45:01Z` 

The `Map.get()` syntax:

greetings,77.07,2023-10-13T02:39:42Z

Note that `Map` does _not_ have an ‘indexer syntax’ like the .NET dictionary. This means that `data['key-two']` will not work and we use `data.get('key-two')` as demonstrated above…

## unpacking the .NET dictionary values

Notice that the values of our `data` are comma-separated strings, three elements:

```plaintext
<greeting>,<number>,<date>
```

Let us unpack these values for use in JavaScript, using the `Array.Map` function [📖 [docs](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Array/map)]:

In [6]:
#!javascript

const json = `
{
    "key-one": "hello,42.3,2023-10-08T04:54:48Z",
    "key-two": "greetings,77.07,2023-10-13T02:39:42Z",
    "key-three": "ohayo,1024,2023-06-26T23:45:01Z"
}`;

const o = JSON.parse(json);

const a = Object.entries(o).map(pair => {
    const k = pair[0];
    const v = pair[1];
    const d = v.split(',');
    if(d.length !== 3) { return [k, { error: 'data was in an unexpected format'}] }
    const greeting = d[0];
    const number = parseFloat(d[1]);
    const date = Date.parse(d[2])
    return [k, {greeting, number, date}];
});

console.log('The array:', a);

const data = new Map(a);

data.forEach((value, key, map) =>
{
    console.log(`map has key \`${key}\` [${map.has(key)}] with value: \`${value}\` `);
});

console.log('The `Map.get()` syntax:', data.get('key-two'));

The array:

map has key `key-one` [true] with value: `[object Object]` 

map has key `key-two` [true] with value: `[object Object]` 

map has key `key-three` [true] with value: `[object Object]` 

The `Map.get()` syntax:

## <!-- -->

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼